# peft qlora chatglm finetune and inference with custom dataset

```
第四周作业：
1、基于 data 目录下的数据训练 ChatGLM3 模型，使用 inference Notebook 对比微调前后的效果。------> this notebook is for this task.
2、（可选）：将 gen_dataset Notebook 改写为 py 文件。
作业代码：
微调的代码：https://github.com/DjangoPeng/LLM-quickstart/blob/main/chatglm/qlora_chatglm3_timestamp.ipynb
使用微调后模型进行推理的代码：https://github.com/DjangoPeng/LLM-quickstart/blob/main/chatglm/chatglm_inference.ipynb
生成数据集的代码：https://github.com/RorschachWwww/LLM-quickstart/blob/main/chatglm/gen_dataset.ipynb

```

In [1]:
# this cell is to train the model with custom dataset from data/zhouyi_dataset_xxx
from datasets import load_dataset, ClassLabel, Sequence
from transformers import AutoTokenizer, TrainingArguments, Trainer
import torch
from peft import TaskType, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING
from transformers import AutoModel, BitsAndBytesConfig
import random
import pandas as pd
from IPython.display import display, HTML
from typing import List, Dict, Optional

# print ptorch info
print(torch.__config__.show(), torch.cuda.get_device_properties(0))

# ======================
# 1. parameneters setup
# ======================

model_name_or_path = 'THUDM/chatglm3-6b'
#train_data_path = 'data/zhouyi_dataset_20240118_163659.csv'
#eval_data_path = None
seed = 8
max_input_length = 512
max_output_length = 1536
lora_rank = 16
lora_alpha = 32
lora_dropout = 0.05
#resume_from_checkpoint = None
prompt_text = ''
compute_dtype = 'bf16'  # fp32 / fp16 / bf16

# support multiple dataset_names to be tested
dataset_names = [
    "data/zhouyi_dataset_20240118_152413.csv",
    "data/zhouyi_dataset_20240118_163659.csv",
    "data/zhouyi_dataset_handmade.csv",
    #"shibing624/AdvertiseGen",
    #can add more dataset for test
]

# use small data for faster test...
num_train_samples = 1000
# use 0.1 of train data
num_eval_samples = num_train_samples // 10

# training paramenters setup
per_device_train_batch_size = 8
per_device_eval_batch_size = 8
gradient_accumulation_steps = 1
learning_rate = 1e-3
num_train_epochs = 3
#eval_steps = num_train_samples // (5 * per_device_train_batch_size * gradient_accumulation_steps)
#save_steps = num_train_samples // (3 * per_device_train_batch_size * gradient_accumulation_steps)
#logging_steps = num_train_samples // (15 * per_device_train_batch_size * gradient_accumulation_steps)
logging_steps = 1
eval_steps = 1
save_steps = 10

# dtype mapping
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# the tokenize function
def tokenize_func(example, tokenizer, ignore_label_id=-100):
    question = prompt_text + example['content']
    if example.get('input', None) and example['input'].strip():
        question += f'\n{example["input"]}'
    answer = example['summary']
    q_ids = tokenizer.encode(text=question, add_special_tokens=False)
    a_ids = tokenizer.encode(text=answer, add_special_tokens=False)
    if len(q_ids) > max_input_length - 2:
        q_ids = q_ids[:max_input_length - 2]
    if len(a_ids) > max_output_length - 1:
        a_ids = a_ids[:max_output_length - 1]
    input_ids = tokenizer.build_inputs_with_special_tokens(q_ids, a_ids)
    question_length = len(q_ids) + 2
    labels = [ignore_label_id] * question_length + input_ids[question_length:]
    return {'input_ids': input_ids, 'labels': labels}

# the DataCollator class
class DataCollatorForChatGLM:
    def __init__(self, pad_token_id: int, max_length: int = 2048, ignore_label_id: int = -100):
        self.pad_token_id = pad_token_id
        self.ignore_label_id = ignore_label_id
        self.max_length = max_length

    def __call__(self, batch: List[Dict[str, List]]) -> Dict[str, torch.Tensor]:
        len_list = [len(d['input_ids']) for d in batch]
        batch_max_len = max(len_list)

        input_ids, labels = [], []
        for len_of_d, d in sorted(zip(len_list, batch), key=lambda x: -x[0]):
            pad_len = batch_max_len - len_of_d
            ids = d['input_ids'] + [self.pad_token_id] * pad_len
            label = d['labels'] + [self.ignore_label_id] * pad_len
            if batch_max_len > self.max_length:
                ids = ids[:self.max_length]
                label = label[:self.max_length]
            input_ids.append(torch.LongTensor(ids))
            labels.append(torch.LongTensor(label))
        return {
            'input_ids': torch.stack(input_ids),
            'labels': torch.stack(labels)
        }

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

# function to free up mem    
def freeup_mem():
    print("\n🔧 Releasing GPU ...")
    import torch, gc
    torch.cuda.empty_cache()
    gc.collect()
    #%reset -f
    
# tain/eval for the datasets
for dataset_name in dataset_names:
    print(f"\n" + "="*80)
    print(f"🔍 Train/Eval with dataset: {dataset_name}")
    print("="*80)

    # call to freeup mem    
    freeup_mem()

    # ======================
    # 1. load dataset
    # ======================
    print(f"loading dataset_name:{dataset_name}")
    dataset = load_dataset("csv", data_files=dataset_name)
    # print out dataset
    print(dataset)

    # show random elements
    show_random_elements(dataset["train"], num_examples=5)

    eval_dataset_loaded = None
    if 'validation' in dataset:
        eval_dataset_loaded = dataset['validation']
    else:
        print("⚠️  The dataset doesn't have 'validation' split， not evaluating.")

    # ======================
    # 2. set the used train/eval samples
    # ======================
    if num_train_samples is not None and num_train_samples > 0 \
    and len(dataset['train']) >= num_train_samples:
        print(f"num_train_samples: {num_train_samples}")
        dataset['train'] = dataset['train'].select(range(num_train_samples))
    if eval_dataset_loaded is not None and num_eval_samples is not None \
and num_eval_samples > 0 and len(eval_dataset_loaded) >= num_eval_samples:
        print(f"num_eval_samples: {num_eval_samples}")
        eval_dataset_loaded = eval_dataset_loaded.select(range(num_eval_samples))

    # ======================
    # 3. Tokenize
    # ======================
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True, revision='b098244')

    column_names = dataset['train'].column_names
    tokenized_train = dataset['train'].map(
        lambda x: tokenize_func(x, tokenizer), 
        batched=False, 
        remove_columns=column_names
    )
    tokenized_train = tokenized_train.shuffle(seed=seed)
    tokenized_train = tokenized_train.flatten_indices()


    tokenized_eval = None
    if eval_dataset_loaded is not None:
        column_names_eval = eval_dataset_loaded.column_names
        tokenized_eval = eval_dataset_loaded.map(
            lambda x: tokenize_func(x, tokenizer), 
            batched=False, 
            remove_columns=column_names_eval
        )

        tokenized_eval = tokenized_eval.shuffle(seed=seed)
        tokenized_eval = tokenized_eval.flatten_indices()
    
    # ======================
    # 4. init the model, LoRA、QLoRA
    # ======================
    model = AutoModel.from_pretrained(
        model_name_or_path,
        quantization_config=BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type='nf4',
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=_compute_dtype_map[compute_dtype]
        ),
        device_map='auto',
        trust_remote_code=True,
        revision='b098244'
    )

    model.supports_gradient_checkpointing = True  
    model.gradient_checkpointing_enable()
    model.enable_input_require_grads()

    model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

    kbit_model = prepare_model_for_kbit_training(model)
    target_modules = TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['chatglm']
    lora_config = LoraConfig(
        target_modules=target_modules,
        r=lora_rank,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        bias='none',
        inference_mode=False,
        task_type=TaskType.CAUSAL_LM
    )
    qlora_model = get_peft_model(kbit_model, lora_config)
    qlora_model.print_trainable_parameters()

    # ======================
    # 5. TrainingArguments & Trainer
    # ======================

    # Split the string by '/' and take the last part (the filename)
    # "data/zhouyi_dataset_20240118_163659.csv"
    print(f"dataset_name={dataset_name}")
    filename = dataset_name.split('/')[-1]  # 'zhouyi_dataset_20240118_163659.csv'

    # Split by '_' and take the parts after 'dataset'
    # parts = filename.split('_')
    # Assuming structure: ... dataset YYYYMMDD HHMMSS .csv
    # date_part = parts[-2]  # '20240118'
    # time_part = parts[-1].split('.')[0]  # '163659'
    # timestamp = f"{date_part}_{time_part}"
    # zhouyi_dataset_20240118_163659
    part1 = filename.split('.')[-2]
    saved_dir = f"models/{model_name_or_path}-epoch{num_train_epochs}-{part1}"
    print(f"training_args num_train_epochs: {num_train_epochs}, \
logging_steps: {logging_steps}, eval_steps: {eval_steps}, save_steps: {save_steps}\
, per_device_train_batch_size: {per_device_train_batch_size}, , gradient_accumulation_steps: {gradient_accumulation_steps}")

    training_args = TrainingArguments(
        output_dir=saved_dir,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_eval_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=learning_rate,
        num_train_epochs=num_train_epochs,
        lr_scheduler_type="linear",
        warmup_ratio=0.1,
        evaluation_strategy="steps" if eval_dataset_loaded is not None else "no",
        eval_steps=eval_steps,
        save_strategy="steps",
        save_steps=save_steps,
        logging_steps=logging_steps,
        optim="adamw_torch",
        fp16=True,
    )

    data_collator = DataCollatorForChatGLM(pad_token_id=tokenizer.pad_token_id)
    
    trainer = Trainer(
        model=qlora_model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_eval,
        data_collator=data_collator,
    )

    # ======================
    # 6. train & auto print the train/validation loss
    # ======================
    print(f"start train...")
    trainer.train()
    print(f"end train.")

    # ======================
    # 7. saved the tuned model
    # ======================
    trainer.model.save_pretrained(saved_dir)
    print(f"✅ fine tuned model saved to path: {saved_dir}")

PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.6 (Git Hash 86e6af5974177e513fd3fee58425e1063e7f1361)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90
  - CuDNN 8.9.2
  - Magma 2.6.1
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=12.1, CUDNN_VERSION=8.9.2, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -DNDEBUG -DUSE_KINET

,content,summary
0,需卦在周易中是什么象征？,"需卦""由上卦坎（水）和下卦乾（天）组成，上卦为云，下卦为天。上卦坎为险，下卦乾为刚，象征着云浮聚于天上，待时降雨之象。预示抓到俘虏，大吉大利，有利于涉水渡河。君子观此卦象，可以宴饮安乐，待时而动。\n\n这个卦象表示踌躇期待，虽然刚强，但前面有险阻，应当等待，涉大川则利。时机尚未成熟，需要耐心等待，不宜冒进。谋事应审时度势，耐心等待，切勿冒险，欲速不达。自信、坚守中正，可化险为夷。情况有利时，仍得居安思危。\n\n需卦的核心哲学是谋事应审时度势，耐心等待，切勿冒险，欲速不达。自信、坚守中正，可以化险为夷。需要控制饮食，保持健康的生活方式。在感情和婚姻方面，需要慎重、诚实、和柔克刚。\n\n在行动之初，需要极大的耐心，观时待变，创造条件和机会。等待时机成熟后，大器必定晚成。在事业发展中，遇到困难和险阻时，必须十分谨慎，坦然对待小人的中伤。为人处事应谦和、坦率，多有他人相助，促使事业成功。当时机成熟后，则必然一帆风顺。""\n\nsource:""《易经》"
1,请解释一下比卦。,"比卦""是周易卦象中的一枚卦，由下卦坤（地）上卦坎（水）组成，预示着吉利的变化。在卜筮时，再次占卜依然吉利，预示长期稳定无灾祸。然而，当不愿臣服的邦国未能前来朝贺时，将会带来危险。\n\n在《象辞》中，比卦被描述为地上有水的情景，反映了相亲相依相互依赖的意义。先王观此卦象，取法于水附大地，地纳江河之象，因此此卦被解释为建立万国，亲近诸侯。\n\n北宋易学家邵雍解释认为，比卦代表水在地面上流动，人际关系亲密和睦，各种事情无忧无虑。\n\n台湾国学大儒傅佩荣解释认为，比卦的时运是众人相贺，财运是善人相扶，家宅方面是长久美满，身体方面则需早求治心腹水肿。\n\n传统解卦认为，比卦预示着相亲相辅，宽宏无私，精诚团结的道理。在运势上，表示平顺，能得贵人提拔，事业宜速战速决，不宜过度迟疑。对于事业、经商、求名、婚恋等方面都有积极的影响，提醒人们待人宽厚、正直，主动热情，并谨慎选择朋友。\n\n总之，""比卦"
2,"""讼卦在周易中怎样表达教育的概念？",在周易中，讼卦是一个极具深意的卦象。上卦为乾（天），下卦为坎（水），两者相背而行，代表天与水违行的状况，象征着事理乖舛和争讼之象。讼卦中有利可图，但必须警惕戒惧，事情中间吉利，但最终会有凶险。在卜卦时，利于会见贵族王公，但不利于涉水渡河。\n\n讼卦的核心哲学是：开始可能顺利，有所收获，但随后会遇到困难和挫折。因此，务必慎之又慎，不得固执已见，避免介入诉讼纠纷的争执之中。退让而不固执，求得化解，安于正理，可免除意外之灾。陷入争讼，即使获胜，最后还得失去，得不偿失。\n\n讼卦的经商指引是：和气生财，吃亏是福，切勿追求不义之财。在商业谈判中要坚持公正、公平、互利的原则，尽量避免发生冲突。\n\n对于决策，讼卦提醒我们，争强好胜，不安于现状，为改变命运和超越他人而奋斗。但缺乏持之以恒的毅力，容易得罪他人，带来诉讼之灾。因此，接受教训，引以为戒，可功成名就。\n\n讼卦所蕴含的智慧是：在面对争端和异见时，要善于退让和求和，坚守正道，谨慎处事，以避免不必要的冲突和损失。
3,"周易的""乾卦讲述了什么？","""乾卦""\nsummary: ""《易经》中的乾卦是六十四卦中的首卦，象征天，由六个阳爻组成，代表着刚健强劲的特性。其卦辞为“元、亨、利、贞”，预示着吉祥如意，同时也教导人们遵守天道的德行。乾卦所蕴含的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。""\n\ncomment: ""在传统解卦中，乾卦预示着大吉大利，事业如日中天，但也提醒要警惕盛极必衰的道理。经商方面顺利发展，但要冷静分析形势，坚持商业道德。对于婚恋，尽管阳盛阴衰，但刚柔可相济，最终形成美满结果。总体而言，乾卦代表着充满活力和力量的时机，但也需要保持谦逊谨慎的态度，以应对可能出现的困难。"
4,"""讼卦涉及哪些哲学思想？",在周易中，讼卦是一个极具深意的卦象。上卦为乾（天），下卦为坎（水），两者相背而行，代表天与水违行的状况，象征着事理乖舛和争讼之象。讼卦中有利可图，但必须警惕戒惧，事情中间吉利，但最终会有凶险。在卜卦时，利于会见贵族王公，但不利于涉水渡河。\n\n讼卦的核心哲学是：开始可能顺利，有所收获，但随后会遇到困难和挫折。因此，务必慎之又慎，不得固执已见，避免介入诉讼纠纷的争执之中。退让而不固执，求得化解，安于正理，可免除意外之灾。陷入争讼，即使获胜，最后还得失去，得不偿失。\n\n讼卦的经商指引是：和气生财，吃亏是福，切勿追求不义之财。在商业谈判中要坚持公正、公平、互利的原则，尽量避免发生冲突。\n\n对于决策，讼卦提醒我们，争强好胜，不安于现状，为改变命运和超越他人而奋斗。但缺乏持之以恒的毅力，容易得罪他人，带来诉讼之灾。因此，接受教训，引以为戒，可功成名就。\n\n讼卦所蕴含的智慧是：在面对争端和异见时，要善于退让和求和，坚守正道，谨慎处事，以避免不必要的冲突和损失。


⚠️  The dataset doesn't have 'validation' split， not evaluating.


/root/miniconda3/envs/byenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

...

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


trainable params: 3,899,392 || all params: 6,247,483,392 || trainable%: 0.06241540401681151
dataset_name=data/zhouyi_dataset_handmade.csv
training_args num_train_epochs: 3, logging_steps: 1, eval_steps: 1, save_steps: 10, per_device_train_batch_size: 8, , gradient_accumulation_steps: 1
start train...


/root/miniconda3/envs/byenv/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,4.173000
2,4.173000
3,3.784700


end train.
✅ fine tuned model saved to path: models/THUDM/chatglm3-6b-epoch3-zhouyi_dataset_handmade


In [4]:
# this cell is to evaluate the modles
# function to free up mem and remove all variables
def freeup_mem_deep():
    print("\n🔧 Releasing GPU ...")
    import torch, gc
    torch.cuda.empty_cache()
    gc.collect()
    %reset -f
# call to freeup mem    
freeup_mem_deep()

import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import time

# ======================
# 1. parameters setup
# ======================

# the base model
model_name_or_path = 'THUDM/chatglm3-6b'

# support multiple fine tuned models
#"data/zhouyi_dataset_20240118_152413.csv",
#"data/zhouyi_dataset_20240118_163659.csv",
#"data/zhouyi_dataset_handmade_20250811_163200.csv",
peft_model_paths = [
    f"models/{model_name_or_path}-epoch3-zhouyi_dataset_20240118_152413",
    f"models/{model_name_or_path}-epoch3-zhouyi_dataset_20240118_163659",
    f"models/{model_name_or_path}-epoch3-zhouyi_dataset_handmade",
    #can add more models for test
]

_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# init q_config（with 4bit）
q_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=_compute_dtype_map['bf16']
)

# ======================
# 2. load the base model
# ======================

print(f"🔍 Loading the base model: {model_name_or_path} (not fine tuned)...")
base_model = AutoModel.from_pretrained(
    model_name_or_path,
    quantization_config=q_config,
    trust_remote_code=True,
    device_map='auto',
    revision='b098244'
)
base_model.requires_grad_(False)

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True, revision='b098244')

# ======================
# 3. function to evaluate and compare the models
# ======================
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    print("\n" + "="*80)
    print(f"📊 Comparing result for query: {query}")
    print("="*80)
    #print(f" 📝 Question：{query} \
    print(f"📝Base model response:\n{base_response} \
    \n👉Fine tuned model ({training_tag}) response: \n{ft_response}")
    #print("="*60)
    return base_response, ft_response

# ======================
# 4. Evaluate and compare base model and peft model
# ======================
for peft_model_dir in peft_model_paths:
    print("\n" + "="*80)
    print(f"🔍 Loading PEFT model from: {peft_model_dir}")
    print("="*80)
    
    try:
        # Load the PEFT adapter with the base_model
        config = PeftConfig.from_pretrained(peft_model_dir)
        qlora_model = PeftModel.from_pretrained(base_model, peft_model_dir)
        base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, peft_model_dir)
        base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, peft_model_dir)
        base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, peft_model_dir)
        base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model, peft_model_dir)
        base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model, peft_model_dir)
    except Exception as e:
        print(f"❌ Failed to load the model: {peft_model_dir} with error: {e}")
    


🔧 Releasing GPU ...
🔍 Loading the base model: THUDM/chatglm3-6b (not fine tuned)...


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]


🔍 Loading PEFT model from: models/THUDM/chatglm3-6b-epoch3-zhouyi_dataset_20240118_152413

📊 Comparing result for query: 解释下乾卦是什么？
📝Base model response:
乾卦是周易中的一卦，代表天，象征着刚健强劲的特性。它由两个阳爻夹一个阴爻构成，象征着阳刚之气的聚集。乾卦象征着刚健、强盛、独立自主的特性，代表着锐利、直率、果敢的性格。在《易经》中，乾卦预示着健旺、强盛、独立自主的情况，并带来吉兆。

乾卦的卦辞描述了阳爻在前的特性，强调刚健、锐利和果敢，并预示着可能带来成功。同时，需要注意准备好了燃料（前进）才能前进，而阴爻提示需要谨慎行事，以免带来灾难。在决策时，应当积极刚健，而谨慎柔弱，以事业成功为吉，否事业失败为凶。

在古代，人们认为乾卦代表天，象征刚健、强盛、独立自主的特性。在决策时，应该积极刚健，而谨慎柔弱，以事业成功为吉，否事业失败为凶。同时，经商求谋须谨慎选择方向，同时注意合作，以利益最大化为目标。经商求谋方面，须得遇贤人，根据时势变化，抓住机遇。在事业中，需要积极刚健，同时谨慎柔弱，以事业成功为吉，否事业失败为凶。

乾卦的核心哲学是：刚健强劲，阳刚之气的聚集，代表着锐利、直率、果敢的性格。在决策时，应该积极刚健，而谨慎柔弱，以事业成功为吉，否事业失败为凶。     
👉Fine tuned model (models/THUDM/chatglm3-6b-epoch3-zhouyi_dataset_20240118_152413) response: 
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，象征刚健强劲的特性。它由两个乾卦叠加而成，代表着刚健强劲的特性。在卜问中，乾卦预示着吉祥如意，同时也提醒面临着修养和成长的过程。在决策中，意味着刚健强劲的特性会带来吉祥如意，但需要警惕可能会带来困难。

乾卦的特性包括：

1. 吉祥：乾卦预示着事情顺利，带来吉祥如意。
2. 刚健：乾卦代表刚健强劲的特性，表示事情的发展需要刚健强劲的毅力。
3. 突出：乾卦强调刚健强劲的特性，具有突出和领导能力。
4. 养：乾卦意味着养成了刚健强劲的特性，能够养成了坚定的品德。

 summary:

乾卦 